load libraries

In [18]:
%load_ext autoreload
%autoreload 2
from synthpop.census_helpers import Census
from synthpop import categorizer as cat
from synthpop import synthesizer
from synthpop.recipes import starter
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', 500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


api key for census data

In [19]:
c = Census('7a25a7624075d46f112113d33106b6648f42686a')

Identify the fields we want at tract and block-group level

In [20]:
#Households
income_columns = ['B19001_0%02dE'%i for i in range(1, 18)]
vehicle_columns = ['B08201_0%02dE'%i for i in range(1, 7)]
workers_columns = ['B08202_0%02dE'%i for i in range(1, 6)]
families_columns = ['B11001_001E', 'B11001_002E']
# year_built_columns= ['B25034_001E', 'B25034_002E', 'B25034_003E'] # includes vacant structures?
tenure_columns=['B25063_001E', 'B25075_001E']
block_group_columns = income_columns + families_columns + tenure_columns
tract_columns = vehicle_columns + workers_columns

#Persons
population = ['B01001_001E']
sex = ['B01001_002E', 'B01001_026E']
# race = ['B02001_0%02dE'%i for i in range(1,11)]
male_age_columns = ['B01001_0%02dE'%i for i in range(3,26)]
female_age_columns = ['B01001_0%02dE'%i for i in range(27,50)]
# worker_class_columns=['B08128_0%02dE'%i for i in range(1, 7)]
worker_class_columns=['B24080_0%02dE'%i for i in [1,3,6,7,8,9,10,13,16,17,18,19,20]]
# One more column needed for the 16 and under not incuded in the worker_class population
all_columns = population + sex + male_age_columns + female_age_columns 
# +worker_class_columns

identify which counties we want data for

In [21]:
state='25'
counties=['017', '025']

For each county, get the person and HH data for this tract and all the BGs in the tract

In [22]:
h_acs=pd.DataFrame()
p_acs=pd.DataFrame()
for county in counties:
    print(county)
    h_acs_c = c.block_group_and_tract_query(block_group_columns,
                    tract_columns, state, county, 
                    merge_columns=['tract', 'county', 'state'],
                    block_group_size_attr="B11001_001E",
                    tract_size_attr="B08201_001E",
                    tract=None, year=2017)
    h_acs=pd.concat([h_acs, h_acs_c])
    p_acs_c = c.block_group_query(all_columns, state, county, tract=None, year=2017)
    p_acs=pd.concat([p_acs, p_acs_c])
h_acs=h_acs.reset_index(drop=True)
p_acs=p_acs.reset_index(drop=True)


017
025


In [23]:
# add another category to the worker_class for under-16s not included
# p_acs['B24080_not_employed']=p_acs.apply(lambda row: row['B01001_001E']-row['B24080_001E'], axis=1)

In [24]:
h_acs.head()

,B11001_001E,B11001_002E,B19001_001E,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E,B25063_001E,B25075_001E,NAME,block group,county,state,tract,B08201_001E,B08201_002E,B08201_003E,B08201_004E,B08201_005E,B08201_006E,B08202_001E,B08202_002E,B08202_003E,B08202_004E,B08202_005E
0,703,432,703,41,0,9,30,18,0,77,95,20,16,19,110,33,87,81,67,370,333,"Block Group 4, Census Tract 3686, Middlesex Co...",4,017,25,368600,703,76,348,227,32,17,703,119,288,229,65
1,657,204,657,125,79,8,0,99,18,0,11,0,26,77,46,75,77,0,16,477,180,"Block Group 3, Census Tract 3686, Middlesex Co...",3,017,25,368600,657,71,325,212,30,16,657,111,269,214,61
2,291,151,291,0,0,0,23,0,0,30,0,0,44,24,65,40,18,22,25,192,99,"Block Group 1, Census Tract 3686, Middlesex Co...",1,017,25,368600,291,31,144,94,13,7,291,49,119,95,27
3,234,91,234,43,0,0,0,0,0,0,0,17,0,35,30,16,66,9,18,157,77,"Block Group 2, Census Tract 3686, Middlesex Co...",2,017,25,368600,234,25,115,75,10,5,234,39,95,76,21
4,558,232,558,16,0,46,0,0,64,27,31,0,13,50,74,76,103,38,20,359,199,"Block Group 5, Census Tract 3686, Middlesex Co...",5,017,25,368600,558,60,276,180,26,13,558,95,228,182,51


add pumas

In [25]:
h_acs['puma'] = h_acs.apply(lambda row: c.tract_to_puma(row['state'], row['county'], row['tract'])[0], axis=1)
p_acs['puma'] = p_acs.apply(lambda row: c.tract_to_puma(row['state'], row['county'], row['tract'])[0], axis=1)

Create the categorised versions

In [26]:
# Households
h_acs_cat = cat.categorize(h_acs, {
#     ("households", "total"): "B11001_001E",
    ("children", "yes"): "B11001_002E",
    ("children", "no"): "B11001_001E - B11001_002E",
    ("income", "lt35"): "B19001_002E + B19001_003E + B19001_004E + "
                        "B19001_005E + B19001_006E + B19001_007E",
    ("income", "gt35-lt100"): "B19001_008E + B19001_009E + "
                        "B19001_010E + B19001_011E + B19001_012E"
                        "+ B19001_013E",
    ("income", "gt100"): "B19001_014E + B19001_015E + B19001_016E"
                        "+ B19001_017E",
    ("cars", "none"): "B08201_002E",
    ("cars", "one"): "B08201_003E",
    ("cars", "two or more"): "B08201_004E + B08201_005E + B08201_006E",
    ("workers", "none"): "B08202_002E",
    ("workers", "one"): "B08202_003E",
    ("workers", "two or more"): "B08202_004E + B08202_005E",
    ("tenure", "owned"): "B25063_001E",
    ("tenure", "rented"): "B25075_001E"
}, index_cols=['NAME'])
h_acs_cat.head()

cat_name                                           cars                   \
cat_value                                          none  one two or more   
NAME                                                                       
Block Group 4, Census Tract 3686, Middlesex Cou...   76  348         276   
Block Group 3, Census Tract 3686, Middlesex Cou...   71  325         258   
Block Group 1, Census Tract 3686, Middlesex Cou...   31  144         114   
Block Group 2, Census Tract 3686, Middlesex Cou...   25  115          90   
Block Group 5, Census Tract 3686, Middlesex Cou...   60  276         219   

cat_name                                           children      income  \
cat_value                                                no  yes  gt100   
NAME                                                                      
Block Group 4, Census Tract 3686, Middlesex Cou...      271  432    268   
Block Group 3, Census Tract 3686, Middlesex Cou...      453  204    168   
Block Group 1, Census Tract 3686, Middlesex Cou...      140  151    105   
Block Group 2, Census Tract 3686, Middlesex Cou...      143   91    109   
Block Group 5, Census Tract 3686, Middlesex Cou...      326  232    237   

cat_name                                                           tenure  \
cat_value                                          gt35-lt100 lt35  owned   
NAME                                                                        
Block Group 4, Census Tract 3686, Middlesex Cou...        337   98    370   
Block Group 3, Census Tract 3686, Middlesex Cou...        160  329    477   
Block Group 1, Census Tract 3686, Middlesex Cou...        163   23    192   
Block Group 2, Census Tract 3686, Middlesex Cou...         82   43    157   
Block Group 5, Census Tract 3686, Middlesex Cou...        195  126    359   

cat_name                                                  workers       \
cat_value                                          rented    none  one   
NAME                                                                     
Block Group 4, Census Tract 3686, Middlesex Cou...    333     119  288   
Block Group 3, Census Tract 3686, Middlesex Cou...    180     111  269   
Block Group 1, Census Tract 3686, Middlesex Cou...     99      49  119   
Block Group 2, Census Tract 3686, Middlesex Cou...     77      39   95   
Block Group 5, Census Tract 3686, Middlesex Cou...    199      95  228   

cat_name                                                        
cat_value                                          two or more  
NAME                                                            
Block Group 4, Census Tract 3686, Middlesex Cou...         294  
Block Group 3, Census Tract 3686, Middlesex Cou...         275  
Block Group 1, Census Tract 3686, Middlesex Cou...         122  
Block Group 2, Census Tract 3686, Middlesex Cou...          97  
Block Group 5, Census Tract 3686, Middlesex Cou...         233

In [27]:
# Persons
p_acs_cat = cat.categorize(p_acs, {
#     ("population", "total"): "B01001_001E",
    ("age", "19 and under"): "B01001_003E + B01001_004E + B01001_005E + "
                             "B01001_006E + B01001_007E + B01001_027E + "
                             "B01001_028E + B01001_029E + B01001_030E + "
                             "B01001_031E",
    ("age", "20 to 35"): "B01001_008E + B01001_009E + B01001_010E + "
                         "B01001_011E + B01001_012E + B01001_032E + "
                         "B01001_033E + B01001_034E + B01001_035E + "
                         "B01001_036E",
    ("age", "35 to 60"): "B01001_013E + B01001_014E + B01001_015E + "
                         "B01001_016E + B01001_017E + B01001_037E + "
                         "B01001_038E + B01001_039E + B01001_040E + "
                         "B01001_041E",
    ("age", "above 60"): "B01001_018E + B01001_019E + B01001_020E + "
                         "B01001_021E + B01001_022E + B01001_023E + "
                         "B01001_024E + B01001_025E + B01001_042E + "
                         "B01001_043E + B01001_044E + B01001_045E + "
                         "B01001_046E + B01001_047E + B01001_048E + "
                         "B01001_049E", 
#     ("race", "white"):   "B02001_002E",
#     ("race", "black"):   "B02001_003E",
#     ("race", "asian"):   "B02001_005E",
#     ("race", "other"):   "B02001_004E + B02001_006E + B02001_007E + "
#                          "B02001_008E",
    ("sex", "male"):     "B01001_002E",
    ("sex", "female"):   "B01001_026E",
#     ("worker_class", "private_for_profit"): "B24080_003E+ B24080_013E",
#     ("worker_class", "private_non_profit"): "B24080_006E+ B24080_016E",
#     ("worker_class", "government"): "B24080_007E+ B24080_008E + B24080_009E"
#                                     "B24080_017E+ B24080_018E + B24080_019E",
#     ("worker_class", "self_employed"): "B24080_010E+ B24080_020E",
#     ("worker_class", "not_employed"): "B24080_not_employed + B24080_011E + B24080_021E"
    
}, index_cols=['NAME'])
p_acs_cat.head()

cat_name                                                    age           \
cat_value                                          19 and under 20 to 35   
NAME                                                                       
Block Group 4, Census Tract 3686, Middlesex Cou...          453      538   
Block Group 3, Census Tract 3686, Middlesex Cou...          250      657   
Block Group 1, Census Tract 3686, Middlesex Cou...          132      242   
Block Group 3, Census Tract 3834, Middlesex Cou...          521      212   
Block Group 2, Census Tract 3834, Middlesex Cou...          346      403   

cat_name                                                                sex  \
cat_value                                          35 to 60 above 60 female   
NAME                                                                          
Block Group 4, Census Tract 3686, Middlesex Cou...      606      250    856   
Block Group 3, Census Tract 3686, Middlesex Cou...      260      104    660   
Block Group 1, Census Tract 3686, Middlesex Cou...      235       40    316   
Block Group 3, Census Tract 3834, Middlesex Cou...      731      132    658   
Block Group 2, Census Tract 3834, Middlesex Cou...      646      250    998   

cat_name                                                 
cat_value                                          male  
NAME                                                     
Block Group 4, Census Tract 3686, Middlesex Cou...  991  
Block Group 3, Census Tract 3686, Middlesex Cou...  611  
Block Group 1, Census Tract 3686, Middlesex Cou...  333  
Block Group 3, Census Tract 3834, Middlesex Cou...  938  
Block Group 2, Census Tract 3834, Middlesex Cou...  647

Define the pums data we want and some functions for converting to the same categories as the ACS data

In [28]:
#Person
def age_cat(r):
    if r.AGEP <= 19: return "19 and under"
    elif r.AGEP <= 35: return "20 to 35"
    elif r.AGEP <= 60: return "35 to 60"
    return "above 60"

def race_cat(r):
    if r.RAC1P == 1: return "white"
    elif r.RAC1P == 2: return "black"
    elif r.RAC1P == 6: return "asian"
    return "other"

def sex_cat(r):
    if r.SEX == 1: return "male"
    return "female"

def tenure_cat(r):
    if r.TEN ==1 or r.TEN ==2: return "owned"
    elif r.TEN ==3:  return "rented"
    return "other"

#Household
def cars_cat(r):
    if r.VEH == 0: return "none"
    elif r.VEH == 1: return "one"
    return "two or more"

def children_cat(r):
    if r.NOC > 0: return "yes"
    return "no"

def income_cat(r): # the example file used FINCP (family income) but the ACS control is Household income
    if r.HINCP > 100000: return "gt100"
    elif r.HINCP > 35000: return "gt35-lt100"
    return "lt35"

def workers_cat(r):
    if r.WIF == 3: return "two or more"
    elif r.WIF == 2: return "two or more"
    elif r.WIF == 1: return "one"
    return "none"

In [29]:
h_pums_cols = ('serialno', 'PUMA00', 'PUMA10',  'NP',
                            'TYPE', 'VEH', 'WIF', 'NOC', 'TEN', 'RNTP', 'BDSP', 'YBL', 'HINCP')
p_pums_cols = ('serialno', 'PUMA00', 'PUMA10', 'AGEP', 'RAC1P', 'SEX', 'SCHL', 'COW')
all_pumas=list(set(h_acs['puma']))

do the synthesis one PUMA at a time

In [35]:
all_households=pd.DataFrame()
all_persons=pd.DataFrame()

for puma in all_pumas:
    print(puma)
    # get the blok groups in this puma
    this_puma_ind=[i for i in range(len(h_acs)) if h_acs.iloc[i]['puma']==puma]
    #download the pums data
    p_pums=c.download_population_pums(state, puma10=puma, usecols=p_pums_cols)
    h_pums=c.download_household_pums(state, puma10=puma, usecols=h_pums_cols)
    print(len(p_pums))
    print(len(h_pums))
    #get the joint distribution of pums data
    h_pums, jd_households = cat.joint_distribution(h_pums,
        cat.category_combinations(h_acs_cat.columns),
        {"cars": cars_cat, "children": children_cat, 
         "income": income_cat, "workers": workers_cat, "tenure": tenure_cat})
    p_pums, jd_persons = cat.joint_distribution(
        p_pums,
        cat.category_combinations(p_acs_cat.columns),
        {"age": age_cat, "sex": sex_cat}
    )
    # simulate households and persons for eacg person in each block-group of this PUMA
    for bg_ind in this_puma_ind:
        zone_name=h_acs_cat.index[bg_ind]
        print(zone_name)
        best_households, best_people, people_chisq, people_p= synthesizer.synthesize(h_acs_cat.iloc[bg_ind].transpose(), p_acs_cat.iloc[bg_ind].transpose(), jd_households, jd_persons, h_pums, p_pums,
                   marginal_zero_sub=.01, jd_zero_sub=.001, hh_index_start=0)
    #     add the puma and bg id to each HH
        best_households['NAME']=zone_name
    #     add people and HH to global list
        all_households=pd.concat([all_households, best_households])
        all_persons=pd.concat([all_persons, best_people])

03400
3215
1175
Block Group 4, Census Tract 3739, Middlesex County, Massachusetts
Drawing 330 households
Block Group 1, Census Tract 3739, Middlesex County, Massachusetts
Drawing 916 households
Block Group 3, Census Tract 3739, Middlesex County, Massachusetts
Drawing 516 households
Block Group 2, Census Tract 3739, Middlesex County, Massachusetts
Drawing 725 households
Block Group 2, Census Tract 3737, Middlesex County, Massachusetts
Drawing 375 households
Block Group 1, Census Tract 3737, Middlesex County, Massachusetts
Drawing 296 households
Block Group 4, Census Tract 3737, Middlesex County, Massachusetts
Drawing 226 households
Block Group 3, Census Tract 3737, Middlesex County, Massachusetts
Drawing 382 households
Block Group 1, Census Tract 3743, Middlesex County, Massachusetts
Drawing 438 households
Block Group 3, Census Tract 3743, Middlesex County, Massachusetts
Drawing 341 households
Block Group 2, Census Tract 3743, Middlesex County, Massachusetts
Drawing 507 households
Block

Block Group 4, Census Tract 3344, Middlesex County, Massachusetts
Drawing 207 households
Block Group 1, Census Tract 3344, Middlesex County, Massachusetts
Drawing 448 households
Block Group 3, Census Tract 3344, Middlesex County, Massachusetts
Drawing 620 households
Block Group 2, Census Tract 3344, Middlesex County, Massachusetts
Drawing 855 households
Block Group 5, Census Tract 3344, Middlesex County, Massachusetts
Drawing 622 households
Block Group 4, Census Tract 3302, Middlesex County, Massachusetts
Drawing 557 households
Block Group 3, Census Tract 3302, Middlesex County, Massachusetts
Drawing 412 households
Block Group 2, Census Tract 3302, Middlesex County, Massachusetts
Drawing 763 households
Block Group 1, Census Tract 3302, Middlesex County, Massachusetts
Drawing 859 households
Block Group 4, Census Tract 3301, Middlesex County, Massachusetts
Drawing 495 households
Block Group 5, Census Tract 3301, Middlesex County, Massachusetts
Drawing 787 households
Block Group 1, Census

Block Group 5, Census Tract 3351, Middlesex County, Massachusetts
Drawing 394 households
Block Group 4, Census Tract 3351, Middlesex County, Massachusetts
Drawing 670 households
Block Group 1, Census Tract 3351, Middlesex County, Massachusetts
Drawing 610 households
Block Group 2, Census Tract 3363, Middlesex County, Massachusetts
Drawing 216 households
Block Group 1, Census Tract 3363, Middlesex County, Massachusetts
Drawing 255 households
Block Group 3, Census Tract 3363, Middlesex County, Massachusetts
Drawing 346 households
Block Group 5, Census Tract 3363, Middlesex County, Massachusetts
Drawing 450 households
Block Group 6, Census Tract 3363, Middlesex County, Massachusetts
Drawing 492 households
Block Group 4, Census Tract 3363, Middlesex County, Massachusetts
Drawing 212 households
Block Group 7, Census Tract 3363, Middlesex County, Massachusetts
Drawing 519 households
Block Group 3, Census Tract 3353.02, Middlesex County, Massachusetts
Drawing 420 households
Block Group 1, Cen

Block Group 3, Census Tract 1602, Suffolk County, Massachusetts
Drawing 489 households
Block Group 3, Census Tract 1801.01, Suffolk County, Massachusetts
Drawing 1164 households
Block Group 1, Census Tract 1801.01, Suffolk County, Massachusetts
Drawing 565 households
Block Group 2, Census Tract 1801.01, Suffolk County, Massachusetts
Drawing 635 households
Block Group 1, Census Tract 1705.01, Suffolk County, Massachusetts
Drawing 953 households
Block Group 2, Census Tract 1705.01, Suffolk County, Massachusetts
Drawing 733 households
Block Group 1, Census Tract 1606.01, Suffolk County, Massachusetts
Drawing 964 households
Block Group 2, Census Tract 1606.01, Suffolk County, Massachusetts
Drawing 529 households
Block Group 2, Census Tract 1704, Suffolk County, Massachusetts
Drawing 475 households
Block Group 1, Census Tract 1704, Suffolk County, Massachusetts
Drawing 529 households
Block Group 3, Census Tract 1704, Suffolk County, Massachusetts
Drawing 401 households
Block Group 4, Census

Drawing 0 households
Block Group 1, Census Tract 9811, Suffolk County, Massachusetts
Drawing 0 households
Block Group 4, Census Tract 9811, Suffolk County, Massachusetts
Drawing 96 households
Block Group 3, Census Tract 9811, Suffolk County, Massachusetts
Drawing 0 households
Block Group 2, Census Tract 1401.05, Suffolk County, Massachusetts
Drawing 434 households
Block Group 1, Census Tract 1401.05, Suffolk County, Massachusetts
Drawing 946 households
Block Group 1, Census Tract 9807, Suffolk County, Massachusetts
Drawing 4 households
Block Group 3, Census Tract 1201.05, Suffolk County, Massachusetts
Drawing 284 households
Block Group 1, Census Tract 1201.05, Suffolk County, Massachusetts
Drawing 260 households
Block Group 2, Census Tract 1201.05, Suffolk County, Massachusetts
Drawing 408 households
Block Group 1, Census Tract 1203.01, Suffolk County, Massachusetts
Drawing 547 households
Block Group 2, Census Tract 1203.01, Suffolk County, Massachusetts
Drawing 455 households
Block Gr

...

Block Group 1, Census Tract 3563, Middlesex County, Massachusetts
Drawing 477 households
Block Group 5, Census Tract 3563, Middlesex County, Massachusetts
Drawing 446 households
Block Group 2, Census Tract 3563, Middlesex County, Massachusetts
Drawing 457 households
Block Group 6, Census Tract 3563, Middlesex County, Massachusetts
Drawing 344 households
Block Group 1, Census Tract 3381, Middlesex County, Massachusetts
Drawing 471 households
Block Group 2, Census Tract 3381, Middlesex County, Massachusetts
Drawing 590 households
Block Group 4, Census Tract 3385, Middlesex County, Massachusetts
Drawing 491 households
Block Group 2, Census Tract 3385, Middlesex County, Massachusetts
Drawing 542 households
Block Group 3, Census Tract 3385, Middlesex County, Massachusetts
Drawing 504 households
Block Group 1, Census Tract 3385, Middlesex County, Massachusetts
Drawing 495 households
Block Group 3, Census Tract 3575, Middlesex County, Massachusetts
Drawing 347 households
Block Group 1, Census

Block Group 3, Census Tract 3571, Middlesex County, Massachusetts
Drawing 359 households
Block Group 4, Census Tract 3571, Middlesex County, Massachusetts
Drawing 372 households
Block Group 2, Census Tract 3571, Middlesex County, Massachusetts
Drawing 674 households
Block Group 1, Census Tract 3577, Middlesex County, Massachusetts
Drawing 382 households
Block Group 2, Census Tract 3577, Middlesex County, Massachusetts
Drawing 295 households
Block Group 3, Census Tract 3577, Middlesex County, Massachusetts
Drawing 430 households
Block Group 4, Census Tract 3577, Middlesex County, Massachusetts
Drawing 440 households
Block Group 2, Census Tract 3382, Middlesex County, Massachusetts
Drawing 424 households
Block Group 1, Census Tract 3382, Middlesex County, Massachusetts
Drawing 1127 households
Block Group 2, Census Tract 3703, Middlesex County, Massachusetts
Drawing 752 households
Block Group 3, Census Tract 3703, Middlesex County, Massachusetts
Drawing 297 households
Block Group 4, Censu

Drawing 531 households
Block Group 3, Census Tract 3527, Middlesex County, Massachusetts
Drawing 165 households
Block Group 1, Census Tract 3527, Middlesex County, Massachusetts
Drawing 378 households
Block Group 2, Census Tract 3527, Middlesex County, Massachusetts
Drawing 400 households
Block Group 2, Census Tract 3547, Middlesex County, Massachusetts
Drawing 585 households
Block Group 1, Census Tract 3547, Middlesex County, Massachusetts
Drawing 570 households
Block Group 1, Census Tract 3548, Middlesex County, Massachusetts
Drawing 407 households
Block Group 2, Census Tract 3548, Middlesex County, Massachusetts
Drawing 556 households
Block Group 2, Census Tract 3534, Middlesex County, Massachusetts
Drawing 690 households
Block Group 1, Census Tract 3534, Middlesex County, Massachusetts
Drawing 533 households
Block Group 2, Census Tract 3532, Middlesex County, Massachusetts
Drawing 574 households
Block Group 3, Census Tract 3532, Middlesex County, Massachusetts
Drawing 438 household

Drawing 1086 households
Block Group 1, Census Tract 3426, Middlesex County, Massachusetts
Drawing 330 households
Block Group 2, Census Tract 3426, Middlesex County, Massachusetts
Drawing 338 households
Block Group 3, Census Tract 3426, Middlesex County, Massachusetts
Drawing 964 households
Block Group 1, Census Tract 3424, Middlesex County, Massachusetts
Drawing 684 households
Block Group 4, Census Tract 3424, Middlesex County, Massachusetts
Drawing 945 households
Block Group 3, Census Tract 3424, Middlesex County, Massachusetts
Drawing 172 households
Block Group 2, Census Tract 3424, Middlesex County, Massachusetts
Drawing 472 households
Block Group 5, Census Tract 3424, Middlesex County, Massachusetts
Drawing 382 households
Block Group 1, Census Tract 3503, Middlesex County, Massachusetts
Drawing 425 households
Block Group 3, Census Tract 3503, Middlesex County, Massachusetts
Drawing 361 households
Block Group 2, Census Tract 3503, Middlesex County, Massachusetts
Drawing 352 househol

Block Group 4, Census Tract 3651, Middlesex County, Massachusetts
Drawing 559 households
Block Group 2, Census Tract 3651, Middlesex County, Massachusetts
Drawing 665 households
Block Group 3, Census Tract 3651, Middlesex County, Massachusetts
Drawing 867 households
Block Group 1, Census Tract 3651, Middlesex County, Massachusetts
Drawing 525 households
Block Group 2, Census Tract 3652.01, Middlesex County, Massachusetts
Drawing 411 households
Block Group 1, Census Tract 3652.01, Middlesex County, Massachusetts
Drawing 726 households
Block Group 3, Census Tract 3652.01, Middlesex County, Massachusetts
Drawing 359 households
Block Group 4, Census Tract 3652.01, Middlesex County, Massachusetts
Drawing 469 households
Block Group 1, Census Tract 3652.02, Middlesex County, Massachusetts
Drawing 834 households
Block Group 3, Census Tract 3652.02, Middlesex County, Massachusetts
Drawing 397 households
Block Group 2, Census Tract 3652.02, Middlesex County, Massachusetts
Drawing 403 households


In [36]:
all_households.to_csv('./Boston/clean/synth_households.csv')
all_persons.to_csv('./Boston/clean/synth_persons.csv')

In [45]:
# all_households.head(100)
all_persons.loc[all_persons['serialno']==2012000014090]

,serialno,puma00,puma10,AGEP,COW,SCHL,SEX,RAC1P,age,sex,cat_id,hh_id
1247,2012000014090,-9,3400,3,NaN,1.0,1,1,19 and under,male,109,0
1248,2012000014090,-9,3400,3,NaN,1.0,1,1,19 and under,male,109,1
1249,2012000014090,-9,3400,3,NaN,1.0,1,1,19 and under,male,109,2
1250,2012000014090,-9,3400,3,NaN,1.0,1,1,19 and under,male,109,3
1251,2012000014090,-9,3400,2,NaN,NaN,1,1,19 and under,male,109,0
1252,2012000014090,-9,3400,2,NaN,NaN,1,1,19 and under,male,109,1
1253,2012000014090,-9,3400,2,NaN,NaN,1,1,19 and under,male,109,2
1254,2012000014090,-9,3400,2,NaN,NaN,1,1,19 and under,male,109,3
1255,2012000014090,-9,3400,39,2.0,23.0,2,1,35 to 60,female,112,0
1256,2012000014090,-9,3400,39,2.0,23.0,2,1,35 to 60,female,112,1
